# Reinforcement Learning — Lab 1  
## FrozenLake (Gymnasium) + Tabular Q-learning

**Goal:** get comfortable with *Gymnasium* and implement a simple *tabular Q-learning* agent on **FrozenLake-v1**.

You’ll:
1. Install and import Gymnasium
2. Explore the environment API
3. Implement **epsilon-greedy** action selection
4. Implement **Q-learning**
5. Evaluate success rate and discuss hyperparameters

> Tip: If you make changes and want a clean run, use **Runtime → Restart and run all**.


### Marking / submission (suggested)
- Your completed notebook (`.ipynb`)
- Short answers to the questions in **Part 6**


---

## Part 1 — Setup (Google Colab)
Run the next cell to install dependencies.


In [ ]:
# Install Gymnasium (the maintained successor to OpenAI Gym).
!pip -q install gymnasium

import gymnasium as gym
import numpy as np
import random


---

## Part 2 — FrozenLake basics

FrozenLake is a small **discrete** environment. By default it is **slippery**, meaning actions can have stochastic outcomes.


In [ ]:
env = gym.make("FrozenLake-v1", is_slippery=True)  # try False later
obs, info = env.reset(seed=0)
print("Initial observation:", obs)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)


### 2.1 Taking a step

Gymnasium’s `step` returns: `(obs, reward, terminated, truncated, info)`

- `terminated` means the episode ended due to the task outcome (goal or hole).
- `truncated` means the episode ended due to a time limit or other constraint.


In [ ]:
obs, info = env.reset(seed=0)
action = env.action_space.sample()
next_obs, reward, terminated, truncated, info = env.step(action)

print("action:", action)
print("next_obs:", next_obs)
print("reward:", reward)
print("terminated:", terminated, "truncated:", truncated)


---

## Part 3 — Epsilon-greedy policy helper

Implement a function that chooses:
- with probability `epsilon`: a **random** action
- otherwise: the **greedy** action `argmax_a Q[s, a]`

Fill in the TODOs.


In [ ]:
def epsilon_greedy_action(Q: np.ndarray, state: int, epsilon: float, n_actions: int) -> int:
    """Choose an action using epsilon-greedy strategy."""
    # TODO:
    # - With prob epsilon, return a random action in {0,...,n_actions-1}
    # - Otherwise return greedy action argmax_a Q[state, a]
    #
    # Hints:
    #   if np.random.random() < epsilon:
    #       return np.random.randint(n_actions)
    #   return int(np.argmax(Q[state]))
    raise NotImplementedError("TODO: implement epsilon_greedy_action")


Quick sanity check: once you implement `epsilon_greedy_action`, the cell below should run.

Expected behaviour:
- with `epsilon=1.0`, actions are basically random
- with `epsilon=0.0`, you always take the greedy action


In [ ]:
# Sanity check (run after implementing epsilon_greedy_action)
n_states = env.observation_space.n
n_actions = env.action_space.n
Q_test = np.zeros((n_states, n_actions))
Q_test[0, 2] = 1.0  # make action 2 greedy in state 0

for eps in [1.0, 0.2, 0.0]:
    actions = [epsilon_greedy_action(Q_test, state=0, epsilon=eps, n_actions=n_actions) for _ in range(20)]
    print(f"epsilon={eps}: {actions}")


---

## Part 4 — Q-learning on FrozenLake

**Q-learning update:**

$$
Q[s,a] \leftarrow Q[s,a] + \alpha \left(r + \gamma \max_{a'} Q[s',a'] - Q[s,a]\right)
$$

We’ll track:
- episode return (sum of rewards)
- success rate (how often we reach the goal)


### 4.1 Training scaffold (fill in TODOs)

Notes:
- FrozenLake rewards are usually `0` except reaching the goal gives `1`.
- Training can require many episodes when `is_slippery=True`.


In [ ]:
from dataclasses import dataclass

@dataclass
class TrainStats:
    returns: list
    successes: list  # 1 if goal reached, else 0

def train_q_learning(
    env_id: str = "FrozenLake-v1",
    episodes: int = 20000,
    alpha: float = 0.1,
    gamma: float = 0.99,
    epsilon_start: float = 1.0,
    epsilon_end: float = 0.05,
    epsilon_decay_fraction: float = 0.8,
    is_slippery: bool = True,
    seed: int = 0,
):
    """Train a tabular Q-learning agent on FrozenLake. Returns (Q, stats)."""
    env = gym.make(env_id, is_slippery=is_slippery)
    n_states = env.observation_space.n
    n_actions = env.action_space.n

    np.random.seed(seed)
    random.seed(seed)

    Q = np.zeros((n_states, n_actions), dtype=np.float32)
    stats = TrainStats(returns=[], successes=[])

    decay_episodes = max(1, int(episodes * epsilon_decay_fraction))

    def epsilon_for_episode(ep: int) -> float:
        if ep >= decay_episodes:
            return epsilon_end
        # TODO: linear decay from epsilon_start to epsilon_end over [0, decay_episodes)
        # One way:
        #   frac = ep / decay_episodes
        #   return epsilon_start + frac * (epsilon_end - epsilon_start)
        raise NotImplementedError("TODO: implement epsilon schedule")

    for ep in range(episodes):
        state, info = env.reset(seed=seed + ep)
        done = False
        ep_return = 0.0

        while not done:
            eps = epsilon_for_episode(ep)
            action = epsilon_greedy_action(Q, state, eps, n_actions)

            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            # TODO: Q-learning update
            # If done: target = reward
            # Else:    target = reward + gamma * np.max(Q[next_state])
            # Then:
            #   Q[state, action] += alpha * (target - Q[state, action])
            raise NotImplementedError("TODO: implement Q-learning update")

            state = next_state
            ep_return += reward

        stats.returns.append(ep_return)
        stats.successes.append(1 if ep_return > 0 else 0)

    env.close()
    return Q, stats


### 4.2 Run training (after TODOs are implemented)

Try:
- `episodes=20000` (slippery) then increase if needed
- or set `is_slippery=False` to see a much easier problem


In [ ]:
# Run training (this will work after you fill in the TODOs above)
Q, stats = train_q_learning(
    episodes=20000,
    alpha=0.1,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.05,
    epsilon_decay_fraction=0.8,
    is_slippery=True,
    seed=0,
)

print("Training finished.")
print("Final 1000-episode average success rate:", float(np.mean(stats.successes[-1000:])))


---

## Part 5 — Evaluation

We evaluate with `epsilon=0` (pure greedy). We'll run a number of episodes and compute:
- average return
- success rate


In [ ]:
def evaluate_greedy_policy(Q: np.ndarray, env_id: str="FrozenLake-v1", episodes: int=200, is_slippery: bool=True, seed: int=123):
    env = gym.make(env_id, is_slippery=is_slippery)
    successes = 0
    returns = []

    for ep in range(episodes):
        state, info = env.reset(seed=seed + ep)
        done = False
        ep_return = 0.0

        while not done:
            action = int(np.argmax(Q[state]))  # greedy
            state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            ep_return += reward

        returns.append(ep_return)
        if ep_return > 0:
            successes += 1

    env.close()
    return float(np.mean(returns)), successes / episodes

avg_return, success_rate = evaluate_greedy_policy(Q, episodes=200, is_slippery=True)
print("Average return:", avg_return)
print("Success rate:", success_rate)


### 5.1 Plot a learning curve

The plot below shows a moving average of success rate.


In [ ]:
import matplotlib.pyplot as plt

window = 200
successes = np.array(stats.successes, dtype=np.float32)
moving = np.convolve(successes, np.ones(window)/window, mode="valid")

plt.figure()
plt.plot(moving)
plt.xlabel("Episode")
plt.ylabel(f"Success rate (moving avg, window={window})")
plt.title("FrozenLake Q-learning training progress")
plt.show()


---

## Part 6 — Questions (write brief answers below)

1. What happens to success rate when you increase vs decrease `epsilon`?
2. What happens when you switch `is_slippery=True` to `False`?
3. Which hyperparameter mattered most for you: `alpha`, `gamma`, the epsilon schedule, or episode count?


### Answers

1.  
2.  
3.  


---

## Optional extension — Run one greedy episode

This cell runs one greedy episode and prints the visited state indices.


In [ ]:
def run_one_episode_greedy(Q: np.ndarray, env_id: str="FrozenLake-v1", is_slippery: bool=True, seed: int=999):
    env = gym.make(env_id, is_slippery=is_slippery)
    state, info = env.reset(seed=seed)
    done = False
    states = [state]
    total = 0.0

    while not done:
        action = int(np.argmax(Q[state]))
        state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        states.append(state)
        total += reward

    env.close()
    return states, total

states, total = run_one_episode_greedy(Q, is_slippery=True)
print("Visited states:", states)
print("Total reward:", total)
